In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import qutip as qt
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from scipy.optimize import NonlinearConstraint
from tqdm import tqdm
from matplotlib.cm import get_cmap
from scipy.optimize import minimize
from scipy.stats import pearsonr
from scipy.stats import wasserstein_distance
from scipy.special import binom
import scipy.interpolate as interp
import itertools
from itertools import chain, combinations, permutations
import os
import fnmatch
import ast
import re
from toqito.matrix_props import is_unitary
# import networkx as nx
# from networkx.generators.random_graphs import erdos_renyi_graph
# from networkx.generators.classic import complete_graph
import matplotlib.pyplot as plt
# import networkx.generators.random_graphs as rg
import scipy.special
from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# rc('text', usetex=True) 

In [8]:
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
I = qt.identity(2)
Had = qt.core.gates.snot()
q0 = qt.basis(2,0)
q1 = qt.basis(2,1)
pl = Had*q0
pi = np.pi
sqrt=np.sqrt

### Function to compute entangling power of unitary gates:
* ent_power_general : calculates contribution for each partition of the system, by providing index/indices that is/are left untraced
* ent_power_total_general : calculates total entangling power a the unitary, i.e. summation over all possible combinations 

Entanglement power paper that I base my considerations: https://doi.org/10.1088/1751-8121/ab749a (link to the journal version), arXiv: 1909.07709, https://arxiv.org/abs/1909.07709 (link to arXiv)

In [4]:
def ent_power_general(U_choi, idx_left):
    NQ = len(U_choi.dims[0][0])
    idx_left = list(idx_left)
    e = 0
    sub_idx = np.arange(NQ, 2*NQ)
    for i in range(NQ+1):
        x_all = combinations(sub_idx,i)
        for x in x_all:
            tr_idx = idx_left + list(x)
            U = U_choi.ptrace(tr_idx)
            e += 2* ((2/3)**NQ)*(U*U).tr()
    return 2 - e

def ent_power_total_general(U, NQ):
    U_choi = qt.to_choi(U)/(2**NQ)
    subsystem_idx = np.arange(NQ)
    EP = 0
    for i in range(1,NQ//2+1):
        q_all = combinations(subsystem_idx, i)
        if NQ % 2 == 0 and i == NQ / 2:
            q_all = list(q_all)
            n_len = len(q_all)
            q_all = q_all[:n_len//2]
        for q in q_all:
            EP += ent_power_general(U_choi, list(q))
    return 1/(2**(NQ-1)-1) * EP

Tangle

In [ ]:
def tangle_full(psi):
    rho = qt.ket2dm(psi)
    p = len(rho.dims[0])
    t = 0
#     labels = list(range(p))
    NQ = len(psi.dims[0])

    subsystem_idx = np.arange(NQ)
    EP = 0
    N = 0
    for i in range(1,NQ//2+1):
        q_all = combinations(subsystem_idx, i)
        if NQ % 2 == 0 and i == NQ / 2:
            q_all = list(q_all)
            n_len = len(q_all)
            q_all = q_all[:n_len//2]
        for q in q_all:
            
            N += 1
            rho_p = rho.ptrace(q)
#             print(f'q: {q}, rho : {rho_p}')
            t += 2*(1-(rho_p*rho_p).tr())

    return 1/N * t

Coherence generating power

In [5]:
def coherence_power(U: np.ndarray) -> float:
    r"""
    Compute the coherence-generating power (CGP) of a unitary operator in the computational basis (https://arxiv.org/abs/1906.09242).
    This function uses 2-norm of coherence as the measure and computes the "extremal" CGP, i.e., averaged over the basis elements
    (as opposed to the average over the entire simplex of incoherent states).

    :raises ValueError: If input matrix is not a unitary.
    :param U: A unitary matrix.
    :return: A value between 0 and 1-1/d that corresponds to the CGP of U.
    """
    
    # extract dimensions of the unitary 
    dim = U.shape[0]
    
    # error if not a unitary
    if is_unitary(U) != True:
        raise ValueError("Not a unitary operator")
    
    # Start computing CGP. X matrix has (i,j) element equal to |U_ij|^2
    X = U * U.conj()
    X = X.real

    # singular values of the realigned unitary; no need to compute the U,V matrices.
    cgp = 1- (np.trace(X @ X.T))/dim
    cgp = cgp.real

    return cgp

### Function to compute maximal value of the EP for a given number of qubits

In [6]:
def max_EP(NQ):
    EP_max = 0
    for j in range(NQ+1):
        ep_max = 0
        for l in range(1,NQ//2+1):
            if l == NQ/2:
                ep_max += binom(NQ,l)*(2**(NQ-abs(l-j))-1)/(2**(NQ-abs(l-j))*2)
            else:
                ep_max += binom(NQ,l)*(2**(NQ-abs(l-j))-1)/(2**(NQ-abs(l-j)))
        EP_max += binom(NQ, j) * ep_max
    EP_ub = 2-2*(2/3)**NQ * (2**NQ-EP_max/(2**(NQ-1)-1))
    return EP_ub

### Function to compute mean value of the EP for a given number of qubits

In [7]:
def mean_EP(NQ):
    return (2**NQ * (2**NQ + 1)-2*3**NQ)/((2**(NQ-1)-1)*(2**NQ+1))